## Auto mode

This is for testing propouses

In [1]:
from config import Color
from helpers import clean  #, local2global
from mavlink.customtypes.location import ENUPose, GRAPose
from plan import Plan
from simulator import (
    QGC,
    ConfigGazebo,
    ConfigQGC,
    ConfigVis,
    Gazebo,
    NoneVisualizer,
    Simulator,
)

clean()

## Simulation Configuration

In [2]:
mission_name = "square"

gra_origin = GRAPose(-35.3633280, 149.1652241, 0, 90)   # east, north, up, heading
enu_origin = ENUPose(0, 0, gra_origin.alt, gra_origin.heading) 

base_homes= ENUPose.list([  # east, north, up, heading
    (0., 0., 0., 0.),
    (15., 0., 0., 0),
    (-15., -15., 0., 0),
    (-15., 0., 0., 45),
    (5., -20., 0., 0.),
])
n_uavs = len(base_homes)
base_paths = [Plan.create_square_path(side_len=10, alt=5,heading=0) for _ in base_homes]

colors=[
    Color.BLUE,
    Color.GREEN,
    Color.BLACK,
    Color.ORANGE,
    Color.RED,
]

## Gazebo Congiguration

In [3]:
gaz_config = ConfigGazebo(origin = enu_origin,
                          world_path="simulator/gazebo/worlds/runway.world")

for path,home,c in zip(base_paths,base_homes,colors):
    gaz_config.add(base_path=path,base_home=home,color=c)
gaz_config.show()

## QGroundControl Configuration

In [4]:
qgc_config = ConfigQGC(origin = gra_origin)

for path,home,color in zip(base_paths,base_homes,colors):
    qgc_config.add(base_path=path,base_home=home,color=color)
qgc_config.show()

In [5]:
novis_config = ConfigVis[int]()

for i,_ in enumerate(base_homes):
    novis_config.add_vehicle(i)

## Save the missions

In [6]:
qgc_config.save_missions()

## Visualization Parameters

In [7]:
novis=NoneVisualizer(novis_config)

gaz= Gazebo(gaz_config,gra_origin)

qgc=QGC(qgc_config)

## Launch Simulatotion

In [8]:
simulator = Simulator(
	visualizers=[qgc],
	mission_names=[f"square_{i + 1}" for i in range(n_uavs)],
	terminals=False,
	verbose=1,
)
orac = simulator.launch(gcs_sysids={'blue 🟦': [1,2,3,4,5]})

🗺️ QGroundControl launched for 2D visualization — simulation powered by ArduPilot SITL.
🚀 ArduPilot SITL vehicle 1 launched (PID 248768)
🚀 ArduPilot SITL vehicle 2 launched (PID 248770)
🚀 ArduPilot SITL vehicle 4 launched (PID 248774)
🚀 ArduPilot SITL vehicle 3 launched (PID 248772)
🚀 UAV logic for vehicle 1 launched (PID 248776)
🚀 ArduPilot SITL vehicle 5 launched (PID 248775)
🚀 UAV logic for vehicle 2 launched (PID 248777)
🚀 UAV logic for vehicle 5 launched (PID 248783)
🚀 UAV logic for vehicle 4 launched (PID 248778)
🚀 UAV logic for vehicle 3 launched (PID 248779)
🚀 Proxy for vehicle 1 launched (PID 248782)
🔗 UAV logic 1 is connected to Ardupilot SITL vehicle 1
🚀 Proxy for vehicle 2 launched (PID 248788)
🔗 UAV logic 2 is connected to Ardupilot SITL vehicle 2
🚀 Proxy for vehicle 5 launched (PID 248792)
🔗 UAV logic 5 is connected to Ardupilot SITL vehicle 5
🚀 Proxy for vehicle 4 launched (PID 248794)
🔗 UAV logic 4 is connected to Ardupilot SITL vehicle 4
🚀 Proxy for vehicle 3 launched 

bind port 5900 for SERIAL0
SERIAL0 on TCP port 5900
bind port 5870 for SERIAL0
SERIAL0 on TCP port 5870
bind port 5890 for SERIAL0
SERIAL0 on TCP port 5890
bind port 5880 for SERIAL0
SERIAL0 on TCP port 5880
bind port 5860 for SERIAL0
SERIAL0 on TCP port 5860


System id: 3
System id: 5
Connection on serial port 5900
[Errno 111] Connection refused sleeping
Loaded defaults from ../../ardupilot/Tools/autotest/default_params/copter.parm,/home/abeldg/uav-cyber-sim/params/vehicle.parm
Smoothing reset at 0.001
Loaded defaults from ../../ardupilot/Tools/autotest/default_params/copter.parm,/home/abeldg/uav-cyber-sim/params/vehicle.parm
Loaded defaults from ../../ardupilot/Tools/autotest/default_params/copter.parm,/home/abeldg/uav-cyber-sim/params/vehicle.parm
Loaded defaults from ../../ardupilot/Tools/autotest/default_params/copter.parm,/home/abeldg/uav-cyber-sim/params/vehicle.parm
Loaded defaults from ../../ardupilot/Tools/autotest/default_params/copter.parm,/home/abeldg/uav-cyber-sim/params/vehicle.parm
Loaded defaults from ../../ardupilot/Tools/autotest/default_params/copter.parm,/home/abeldg/uav-cyber-sim/params/vehicle.parm
Loaded defaults from ../../ardupilot/Tools/autotest/default_params/copter.parm,/home/abeldg/uav-cyber-sim/params/vehicle.p

bind port 5902 for SERIAL1
SERIAL1 on TCP port 5902
bind port 5903 for SERIAL2
SERIAL2 on TCP port 5903
validate_structures:528: Validating structures
Waiting for internal clock bits to be set (current=0x00)
bind port 5882 for SERIAL1
SERIAL1 on TCP port 5882
bind port 5883 for SERIAL2
SERIAL2 on TCP port 5883
validate_structures:528: Validating structures
Waiting for internal clock bits to be set (current=0x00)
bind port 5892 for SERIAL1
SERIAL1 on TCP port 5892
bind port 5893 for SERIAL2
SERIAL2 on TCP port 5893
validate_structures:528: Validating structures
Waiting for internal clock bits to be set (current=0x00)
bind port 5862 for SERIAL1
SERIAL1 on TCP port 5862
bind port 5863 for SERIAL2
SERIAL2 on TCP port 5863
validate_structures:528: Validating structures
bind port 5872 for SERIAL1
SERIAL1 on TCP port 5872
bind port 5873 for SERIAL2
SERIAL2 on TCP port 5873
Waiting for internal clock bits to be set (current=0x00)
validate_structures:528: Validating structures
Waiting for inter

🔗 UAV logic 3 is connected to Oracle ⚪
✅ Heartbeat received

🚀 Starting Vehicle 3 logic
Vehicle 3: ▶️ Plan Started: 📋 auto
Vehicle 3: ▶️ Action Started: 📤 UPLOAD_MISSION
Vehicle 3: ▶️ Step Started: clear uav missions
🔗 UAV logic 2 is connected to Oracle ⚪
✅ Heartbeat received

🚀 Starting Vehicle 2 logic
Vehicle 2: ▶️ Plan Started: 📋 auto
🔗 UAV logic 4 is connected to Oracle ⚪
✅ Heartbeat received

🚀 Starting Vehicle 4 logic
Vehicle 4: ▶️ Plan Started: 📋 auto
Vehicle 2: ▶️ Action Started: 📤 UPLOAD_MISSION
Vehicle 4: ▶️ Action Started: 📤 UPLOAD_MISSION
Vehicle 2: ▶️ Step Started: clear uav missions
Vehicle 4: ▶️ Step Started: clear uav missions

🚀 Starting Proxy 5
🔗 UAV logic 5 is connected to Oracle ⚪
✅ Heartbeat received

🚀 Starting Vehicle 5 logic
Vehicle 5: ▶️ Plan Started: 📋 auto
Vehicle 5: ▶️ Action Started: 📤 UPLOAD_MISSION
Vehicle 5: ▶️ Step Started: clear uav missions

🚀 Starting Proxy 1
🔗 UAV logic 1 is connected to Oracle ⚪
🚀 GCS blue 🟦 launched (PID 249603)
✅ Heartbeat receiv

## Oracle checking

In [ ]:
while len(orac.conns):
    for sysid in list(orac.conns.keys()):
        if orac.is_plan_done(sysid):
            orac.remove(sysid)

Vehicle 1: ▶️ Action Started: 📤 UPLOAD_MISSION
Vehicle 1: ▶️ Step Started: clear uav missions
🔗 UAV logic 1 is connected
🔗 UAV logic 2 is connected
🔗 UAV logic 3 is connected
🔗 UAV logic 4 is connected
Vehicle 5: ✅ Step Done: clear uav missions
Vehicle 5: ✅ 7 waypoints read.
Vehicle 3: ✅ Step Done: clear uav missions
Vehicle 4: ✅ Step Done: clear uav missions
Vehicle 1: ✅ Step Done: clear uav missions
Vehicle 3: ✅ 7 waypoints read.
Vehicle 2: ✅ Step Done: clear uav missions
Vehicle 4: ✅ 7 waypoints read.
Vehicle 1: ✅ 7 waypoints read.
Vehicle 2: ✅ 7 waypoints read.
SIM_VEHICLE: Waiting for SITL to exit
SIM_VEHICLE: Waiting for SITL to exit
SIM_VEHICLE: Waiting for SITL to exit
SIM_VEHICLE: Waiting for SITL to exit
SIM_VEHICLE: Waiting for SITL to exit
🔗 UAV logic 5 is connected
New connection on SERIAL1
New connection on SERIAL1
New connection on SERIAL1
New connection on SERIAL1
New connection on SERIAL1
Vehicle 5: ✅ Sent mission item 0
Vehicle 5: ✅ Sent mission item 1
Vehicle 3: ✅ Se